# Capital Gain Calculator (FIFO Method)
Adapted from VBA to Python

### Structures
1. Transaction queue
    - HashMap (key is Asset, dict is tuple of two deques)
2. Transactions
    - 6 properties: Timestamp, Asset (eg. BTC), Type (buy or sell), Units, Total Amount ($), IRS ID (eg. Gemi 1)

### Outline
1. Validate transaction log CSV
    - Sort CSV by Timestamp property
    - Ensure valid Type property and corresponding Units property sign
    - Ensure other properties properties
2. Input data from transaction log CSV file into a buy transaction queue and sell transaction queue for each asset (eg. transaction log with BTC and ETH transactions -> BTC-Buy, BTC-Sell, ETH-Buy, Eth-Sell)
3. For each asset, run FIFO algorithm
    - While there are still transactions in the sell transaction queue, match front of sell queue with front of buy queue transaction (verifying buy transaction Timestamp property is before sell transaction)
    - When a match is found, add write buy-sell transaction to output CSV file
    - Update remaining balance (Units and Total Amount property) on buy/sell transaction and/or remove empty transaction(s) from respective queue
    - Run until the sell transaction queue is empty, the remaining buy transaction(s) are the carryover
4. Create summary report

# 1. Read and Validate Transaction Log CSV

In [233]:
import pandas as pd

transactions = pd.read_excel('large_example.xlsx')
transactions = transactions.sort_values(by='Timestamp')

# 2. Input Data into Transaction Queues

In [234]:
from collections import deque

# Create Transaction Queues
asset_map = {}

# Define buy and sell constants
BUY = 0
SELL = 1

for index, row in transactions.iterrows():
    # Create buy and sell deques for each asset
    asset = row['Asset']
    if asset not in asset_map:
        buy_deque = deque()
        sell_deque = deque()
        asset_map[asset] = (buy_deque, sell_deque)
    
    # Add transaction to respective deque
    if row['Type'] == "Buy":
        asset_map[asset][BUY].appendleft(row)
    else:
        asset_map[asset][SELL].appendleft(row)

# 3. Run FIFO Transaction Matching

In [235]:
# Define fieldnames
fieldnames = ['Asset', 'Date Purchased', 'Date Sold', 'Units', 'Sale Price', 'Basis',
              'Gain / Loss', 'IRS ID Buy', 'IRS ID Sell']

fifo = pd.DataFrame(columns = fieldnames)
margin = pd.DataFrame(columns = fieldnames)

# Create dictionary to store short and long term capital gain/loss statistics
year_summary = {}
year_summary['totals'] = {'Year': 'Totals', 'STCG': 0, 'STCL': 0, 'LTCG': 0, 'LTCL': 0, 'Net CG': 0}
volume_summary = {'Gain / Loss': 0, 'Sale Price': 0, 'Basis': 0}

for asset in asset_map:
    
    match = {}
    match['Asset'] = asset

    while asset_map[asset][SELL]:
        sell_tx = asset_map[asset][SELL].pop()
        
        if abs(sell_tx['Units']) < .00000001:
            print("ID Buy:", sell_tx['IRS ID'], ",", "Remaining units:", sell_tx['Units'], "(consider as 0)")
            continue
        
        if not asset_map[asset][BUY]:
            match['IRS ID Buy'] = "MARGIN"
            match['IRS ID Sell'] = sell_tx['IRS ID']
            match['Date Purchased'] = sell_tx['Timestamp']
            match['Date Sold'] = sell_tx['Timestamp']
            match['Units'] = abs(sell_tx['Units'])
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = 0
            match['Gain / Loss'] = match['Sale Price']
            
            match_df = pd.DataFrame([match])
            margin = pd.concat([margin, match_df], ignore_index=True)
        
        else:
            buy_tx = asset_map[asset][BUY].pop()

            # Populate match dictionary
            match['IRS ID Buy'] = buy_tx['IRS ID']
            match['IRS ID Sell'] = sell_tx['IRS ID']
            match['Date Purchased'] = buy_tx['Timestamp']
            match['Date Sold'] = sell_tx['Timestamp']

            if buy_tx['Timestamp'] > sell_tx['Timestamp']:
                asset_map[asset][BUY].append(buy_tx)
                match['IRS ID Buy'] = "MARGIN"
                match['Date Purchased'] = sell_tx['Timestamp']
                match['Units'] = abs(sell_tx['Units'])
                match['Sale Price'] = sell_tx['Total Amount']
                match['Basis'] = 0

                match_df = pd.DataFrame([match])
                margin = pd.concat([margin, match_df], ignore_index=True)

            # Sell transaction units are greater, so empty buy transaction
            elif abs(sell_tx['Units']) > buy_tx['Units']:

                # Calculate pro rata sale price
                pro_rata_sale_price = buy_tx['Units'] / abs(sell_tx['Units']) * sell_tx['Total Amount']

                # Populate match dictionary
                match['Units'] = buy_tx['Units']
                match['Sale Price'] = pro_rata_sale_price
                match['Basis'] = buy_tx['Total Amount']

                # Update sell transaction information and put back into deque
                sell_tx['Units'] = sell_tx['Units'] + buy_tx['Units']
                sell_tx['Total Amount'] = sell_tx['Total Amount'] - pro_rata_sale_price
                asset_map[asset][SELL].append(sell_tx)

            # Buy transaction units are greater, so empty sell transaction
            elif abs(sell_tx['Units']) < buy_tx['Units']:

                # Calculate pro rata basis
                pro_rata_basis = abs(sell_tx['Units']) / buy_tx['Units'] * buy_tx['Total Amount']

                # Populate match dictionary
                match['Units'] = abs(sell_tx['Units'])
                match['Sale Price'] = sell_tx['Total Amount']
                match['Basis'] = pro_rata_basis

                # Update buy transaction information and put back into deque
                buy_tx['Units'] = buy_tx['Units'] + sell_tx['Units']
                buy_tx['Total Amount'] = buy_tx['Total Amount'] - pro_rata_basis
                asset_map[asset][BUY].append(buy_tx)

            # Transaction units are the same
            else:
                # Populate match dictionary
                match['Units'] = buy_tx['Units']
                match['Sale Price'] = sell_tx['Total Amount']
                match['Basis'] = buy_tx['Total Amount']
            
            
            
        
        # Calculate match gain or loss
        match['Gain / Loss'] = match['Sale Price'] - match['Basis']
        
        match_df = pd.DataFrame([match])
        fifo = pd.concat([fifo, match_df], ignore_index=True)
        
        # Update year by year summary statistics
        year = sell_tx['Timestamp'].year
        
        if year not in year_summary:
            year_summary[year] = {'Year': year, 'STCG': 0, 'STCL': 0, 'LTCG': 0, 'LTCL': 0, 'Net CG': 0}
        
        if sell_tx['Timestamp'] - buy_tx['Timestamp'] < pd.Timedelta(365, "d"):
            if match['Gain / Loss'] > 0:
                year_summary[year]['STCG'] += match['Gain / Loss']
                year_summary['totals']['STCG'] += match['Gain / Loss']
            else:
                year_summary[year]['STCL'] += match['Gain / Loss']
                year_summary['totals']['STCL'] += match['Gain / Loss']
        else:
            if match['Gain / Loss'] > 0:
                year_summary[year]['LTCG'] += match['Gain / Loss']
                year_summary['totals']['LTCG'] += match['Gain / Loss']
            else:
                year_summary[year]['LTCL'] += match['Gain / Loss']
                year_summary['totals']['LTCL'] += match['Gain / Loss']
                
        year_summary[year]['Net CG'] += match['Gain / Loss']
        year_summary['totals']['Net CG'] += match['Gain / Loss']
        
        # Update volume and total capital gain statistics
        volume_summary['Gain / Loss'] += match['Gain / Loss']
        volume_summary['Sale Price'] += match['Sale Price']
        volume_summary['Basis'] += match['Basis']
    
    
    # The asset carryover are the remaining transactions in the buy deque
    while asset_map[asset][BUY]:
        buy_tx = asset_map[asset][BUY].pop()
        
        if abs(buy_tx['Units']) < .00000001:
            print("ID Buy:", buy_tx['IRS ID'], ",", "Remaining units:", buy_tx['Units'], "(consider as 0)")
            continue
              
        # Populate match as a carryover
        match['IRS ID Buy'] = buy_tx['IRS ID']
        match['IRS ID Sell'] = '-'
        match['Date Purchased'] = buy_tx['Timestamp']
        match['Date Sold'] = '-'
        match['Units'] = buy_tx['Units']
        match['Sale Price'] = '-'
        match['Basis'] = buy_tx['Total Amount']
        match['Gain / Loss'] = '-'
        
        # Write carryover into output CSV file
        match_df = pd.DataFrame([match])
        fifo = pd.concat([fifo, match_df], ignore_index=True)

# Add volume summary to output
volume_df = pd.DataFrame([volume_summary])
fifo = pd.concat([fifo, volume_df], ignore_index=True)

ID Buy: Gemi 7 , Remaining units: -1.7763568394002505e-15 (consider as 0)
ID Buy: Gemi 976 , Remaining units: 9.62840918106167e-14 (consider as 0)


# 4. Create Summary Report

In [236]:
summary = pd.DataFrame.from_dict(year_summary, orient='index')

with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
    fifo.to_excel(writer, sheet_name='FIFO', startrow = 2, index = False)
    summary.to_excel(writer, sheet_name='Summary', startrow = 2, index = False)
    margin.to_excel(writer, sheet_name='Margin', startrow = 2, index = False)
       
    workbook  = writer.book
    worksheet = writer.sheets['FIFO']
    
    currency_format = workbook.add_format({'num_format': '[$$-en-US]#,##0.00_);[Red]([$$-en-US]#,##0.00)'})
    unit_format = workbook.add_format({'num_format': '0.00000000_);(0.00000000)'})
      
    worksheet.set_column(0, 0, 5)
    worksheet.set_column(1, 2, 20)
    worksheet.set_column('D:D', 16, unit_format)
    worksheet.set_column('E:G', 16, currency_format)
    worksheet.set_column(7, 8, 12)
    
    

    worksheet = writer.sheets['Summary']
    worksheet.set_column(0, 0, 5)
    worksheet.set_column(1, 5, 12, currency_format)
    
    worksheet = writer.sheets['Margin']
    worksheet.set_column(0, 0, 5)
    worksheet.set_column(1, 2, 20)
    worksheet.set_column('D:D', 16, unit_format)
    worksheet.set_column('E:G', 16, currency_format)
    worksheet.set_column(7, 8, 12)